# Universal Pre-processing Notebook

This notebook shows the pre-processing steps for both datasets: 
 - Russian Troll Dataset (Github)
 - Manually pulled Verified Tweets using the Twitter API Academic License

Pre-processing is done separately due to the nature of the datasets and wanting to leave additional features available for EDA.

This notebook assumes that the "Merge_JSON" notebook has already been run to generate a merged json file.

In [13]:
import pandas as pd 
import os
import numpy as np
import demoji
import json

## Troll Tweets CSV Pre-processing

In [7]:
#os.getcwd()
#os.chdir('/Users/katymatulay/Documents/Drexel - Grad School/07 Fall 2022/DSCI591/Capstone/data/')

In [8]:
#load csv for troll dataset (old delete once new dataset is confirmed valid)
#all_data=pd.read_csv("../data/all_tweets_encoded.csv", encoding='UTF-8',low_memory=False)

In [16]:
#load CSV for troll dataset (updated)
all_data=pd.read_csv("../data/csv_snapshot.csv", encoding='UTF-8',low_memory=False)

In [17]:
all_data.shape

(2946207, 22)

In [6]:
pd.set_option('display.max_columns', None)
all_data.head(1)

,Unnamed: 0,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN


In [90]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946207 entries, 0 to 2946206
Data columns (total 22 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Unnamed: 0          int64 
 1   external_author_id  object
 2   author              object
 3   content             object
 4   region              object
 5   language            object
 6   publish_date        object
 7   harvested_date      object
 8   following           int64 
 9   followers           int64 
 10  updates             int64 
 11  post_type           object
 12  account_type        object
 13  retweet             int64 
 14  account_category    object
 15  new_june_2018       int64 
 16  alt_external_id     object
 17  tweet_id            int64 
 18  article_url         object
 19  tco1_step1          object
 20  tco2_step1          object
 21  tco3_step1          object
dtypes: int64(7), object(15)
memory usage: 494.5+ MB


#### Step 1: Filter for English Language Tweets Only (for NLP model)

In [91]:
all_data.language.value_counts()

English                2116867
Russian                 610943
German                   86983
Ukrainian                38669
Italian                  18063
Serbian                   9480
Uzbek                     9334
Bulgarian                 9236
LANGUAGE UNDEFINED        8320
Arabic                    7588
Macedonian                5178
French                    4562
Spanish                   3011
Norwegian                 2237
Farsi (Persian)           1684
Romanian                  1629
Dutch                     1165
Swedish                   1014
Vietnamese                 982
Estonian                   770
Japanese                   764
Catalan                    589
Finnish                    579
Albanian                   572
Croatian                   544
Icelandic                  498
Pushto                     487
Portuguese                 463
Turkish                    445
Lithuanian                 416
Polish                     386
Somali                     276
Slovak  

In [18]:
English_tweets = all_data[all_data['language']=='English']

In [19]:
cols_to_keep = ['external_author_id','author','content','region','language','publish_date','following','followers','updates',
               'retweet','account_category','tweet_id','tco1_step1']

In [20]:
English_tweets = English_tweets[cols_to_keep]

In [21]:
len(English_tweets.columns)

13

In [22]:
#Rename columns to match JSON
English_tweets = English_tweets.rename(columns={"retweet":"is_retweet", "tco1_step1":"full_url"})

In [23]:
#add data_source column to denote "troll"
English_tweets['data_source']= 'Troll'

In [24]:
#write to file for ease of import later
English_tweets.to_csv("../data/English_tweets.csv")

## JSON Tweets Pre-processing

In [15]:
#load merged json dataset from Twitter API (delete once new JSON file is confirmed)
with open('../data/tweet_master.json', encoding='UTF-8') as json_file:
    tweet_master = json.load(json_file)

In [ ]:
#load merged json dataset from Twitter API (too large, delete)
#with open('../data/json_snapshot.json', encoding='UTF-8') as json_file:
#    tweet_master = json.load(json_file)

In [25]:
#convert json to pandas dataframe using normalize to flatten dict
df= pd.json_normalize(tweet_master)

### Step 1: Identify Relevant Cols

In [26]:
df.shape

(493186, 57)

In [20]:
pd.set_option('display.max_columns', None)

In [27]:
df.head(2)

,author_id,id,lang,conversation_id,edit_history_tweet_ids,reply_settings,created_at,context_annotations,source,text,...,author.entities.url.urls,author.entities.description.urls,author.url,author.location,author.entities.description.mentions,entities.cashtags,attachments.poll_ids,geo.coordinates.type,geo.coordinates.coordinates,author.entities.description.hashtags
0,22203756,930982584649175047,en,930982584649175047,[930982584649175047],everyone,2017-11-16T02:15:19.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...",Twitter for iPhone,Great meeting w/ GOP Govs at @The_RGA on @real...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22203756,930954564769124352,en,930954365334097921,[930954564769124352],everyone,2017-11-16T00:23:59.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...",Twitter for iPhone,.@realdonaldtrump's vision for tax cuts will d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.columns

Index(['author_id', 'id', 'lang', 'conversation_id', 'edit_history_tweet_ids',
       'reply_settings', 'created_at', 'context_annotations', 'source', 'text',
       'possibly_sensitive', 'media', 'edit_controls.edits_remaining',
       'edit_controls.is_edit_eligible', 'edit_controls.editable_until',
       'attachments.media_keys', 'entities.annotations', 'entities.mentions',
       'entities.urls', 'entities.hashtags', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'author.description',
       'author.public_metrics.followers_count',
       'author.public_metrics.following_count',
       'author.public_metrics.tweet_count',
       'author.public_metrics.listed_count', 'author.id', 'author.username',
       'author.verified', 'author.created_at', 'author.name',
       'author.pinned_tweet_id', 'author.profile_image_url',
       'author.protected', 'in_reply_to_user_id', 'referenced_tweets',
      

In [28]:
len(df.columns)

57

#### Creating column lists: 1st for EDA and another for merging the datasets

In [29]:
cols_keep_EDA = ['author_id','created_at','id','text','lang','author.location','author.name',
             'public_metrics.retweet_count','public_metrics.reply_count','public_metrics.like_count',
             'public_metrics.quote_count','author.username','author.public_metrics.followers_count',
             'author.public_metrics.following_count','author.entities.url.urls','author.created_at',
             'author.verified','context_annotations','entities.annotations','entities.mentions',
             'entities.hashtags', 'entities.urls']

In [30]:
df1 = df[cols_keep_EDA]

In [31]:
len(df1.columns)

22

In [32]:
df1= df1.copy()

### Step 2: Extract Feature Cols to match Troll Dataset Columns

**Feature: Is_Retweet**

In [33]:
#add new column with binary 1/0 if tweet is Retweet
df1['is_retweet']= np.where(df1.text.str.startswith('RT @'),1,0)

In [34]:
df1['is_retweet'].value_counts()

0    409630
1     83556
Name: is_retweet, dtype: int64

**Feature: Updates**

In [36]:
#cols to add together to match the main dataset column "updates"
update_cols = ['public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count']

In [37]:
df1['updates']= df1[update_cols].sum(axis=1)

In [38]:
df1['updates'].describe()

count    4.931860e+05
mean     5.865469e+02
std      1.469165e+04
min      0.000000e+00
25%      1.000000e+01
50%      4.500000e+01
75%      1.430000e+02
max      5.732199e+06
Name: updates, dtype: float64

**Feature: account_category**

In [39]:
#add Account_type column to represent verified account
df1['account_category']= df1['author.verified'].apply(lambda x: 'Verified_User' if True else 0)

In [40]:
df1['account_category'].value_counts()

Verified_User    493186
Name: account_category, dtype: int64

**Feature: data_source**

In [41]:
#delete later if handled in 01_merge
df1['data_source'] = 'Verified_User'

In [42]:
df1['data_source'].value_counts()

Verified_User    493186
Name: data_source, dtype: int64

### Step 3: Column updates

In [43]:
## change column names to match original dataset for merging
df2=df1.rename(columns={"author_id": "external_author_id", "created_at": "publish_date", "text": "content",
                  "lang": "language", "author.location":"region", "author.username":"author",
                  "author.name":"full_name","author.public_metrics.followers_count": "followers",
                  "author.public_metrics.following_count": "following",
                  "id": "tweet_id","entities.urls":"full_url"}, errors="raise")

In [44]:
df2.columns

Index(['external_author_id', 'publish_date', 'tweet_id', 'content', 'language',
       'region', 'full_name', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'author', 'followers', 'following',
       'author.entities.url.urls', 'author.created_at', 'author.verified',
       'context_annotations', 'entities.annotations', 'entities.mentions',
       'entities.hashtags', 'full_url', 'is_retweet', 'updates',
       'account_category', 'data_source'],
      dtype='object')

In [45]:
#drop columns for merging
df3= df2.drop(columns=['public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count','author.entities.url.urls', 'author.created_at', 'author.verified',
       'context_annotations', 'entities.annotations', 'entities.mentions',
       'entities.hashtags','full_name'])

In [73]:
df3.columns

Index(['external_author_id', 'publish_date', 'tweet_id', 'content', 'language',
       'region', 'author', 'followers', 'following', 'full_url', 'is_retweet',
       'updates', 'account_category', 'data_source'],
      dtype='object')

In [47]:
len(df3.columns)

14

In [48]:
#DELETE after changing data source for JSON
#grab list of users from verified users
user_list = df3['author'].unique()

In [49]:
#DELETE after changing data source for JSON
print(user_list, len(user_list))

['Mike_Pence' 'nytimes' 'TIME' 'thehill' 'business' 'Forbes' 'csmonitor'
 'TheAtlantic' 'WSJ' 'AP' 'BBCWorld' 'abcnews' 'propublica' 'USATODAY'
 'Newsweek' 'NPR' 'Reuters' 'FoxBusiness' 'TheIJR' 'RealClearNews'
 'CondoleezzaRice' 'USArmy' 'smithsonian' 'DeptofDefense' 'USDA' 'fema'
 'FCC' 'USGS' 'PeaceCorps' 'MittRomney' 'DonaldJTrumpJr' 'BarackObama'
 'elonmusk' 'HillaryClinton' 'AskTSA' 'NASA' 'NSF' 'librarycongress'
 'HHSGov' 'FBI' 'StateDept' 'NIH' 'CDCgov' 'US_FDA' 'TheJusticeDept'
 'SarahPalinUSA' 'axios' 'BillGates' 'MichelleObama'] 49


In [50]:
#validate that columns match between datasets
sorted(English_tweets.columns) == sorted(df3.columns)

True

### Step 4: Merging the two datasets

In [51]:
merged = pd.concat([English_tweets,df3], axis=0)
merged.head()

,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url,data_source
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,1052,9636,253,0,RightTroll,914580356430536707,https://twitter.com/10_gop/status/914580356430...,Troll
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,1054,9637,254,0,RightTroll,914621840496189440,https://twitter.com/damienwoody/status/9145685...,Troll
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,1054,9637,255,1,RightTroll,914623490375979008,https://twitter.com/10_gop/status/913231923715...,Troll
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,1062,9642,256,0,RightTroll,914639143690555392,https://twitter.com/10_gop/status/914639143690...,Troll
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,1050,9645,246,1,RightTroll,914312219952861184,https://twitter.com/realDonaldTrump/status/914...,Troll


In [72]:
merged.shape

(2610053, 14)

## Merged DF Pre-Processing

### Step 1: Add new features

**Feature: has_URL**

In [54]:
#Extract hyperlink from text
#add new column with binary 1/0 if tweet is Retweet
merged['has_URL']= np.where(merged.content.str.contains('http'),1,0)

In [55]:
merged.has_URL.value_counts()

1    1765935
0     844118
Name: has_URL, dtype: int64

**Feature: Emoji_text**

In [56]:
''' The following converts a text string with emojis into a list of descriptive text strings.
    Duplicate emojis are captured as each emoji converts to 1 text string.'''
def convert_emoji_list(x):
    lst=[]
    estring = ''
    import demoji
    import numpy as np
    if x is not np.nan:
        #extract list of text from demoji func
        lst = demoji.findall_list(x)
        if len(lst)<0:
            return np.nan
     
        else:
            return(lst)

In [76]:
# TEST FUNCTION FIRST
test = merged[:5]
test['emoji_text']= test['content'].apply(convert_emoji_list)

<ipython-input-76-d9e0eebb0ff4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['emoji_text']= test['content'].apply(convert_emoji_list)


In [77]:
test[4:5]

,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url,data_source,has_URL,emoji_text
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,1050,9645,246,1,RightTroll,914312219952861184,https://twitter.com/realDonaldTrump/status/914...,Troll,1,[flag: United States]


In [57]:
#Delete null tweets
merged['content'].isnull().sum()

1

In [58]:
merged = merged.dropna(subset=['content'])

In [60]:
## Warning this takes a long time to run
merged['emoji_text']= merged['content'].apply(convert_emoji_list)

In [61]:
merged['emoji_text'][:10]

0                                []
1                                []
2                                []
3                                []
4             [flag: United States]
5                                []
6    [honeybee, honeybee, honeybee]
7                                []
8                                []
9                                []
Name: emoji_text, dtype: object

**Feature: Emoji_count**

In [65]:
#add another feature column: # emojis used in tweet
merged['emoji_count']= merged['emoji_text'].apply(lambda x: len(x))

In [66]:
merged['emoji_count'].describe()

count    2.610052e+06
mean     4.487114e-02
std      4.770693e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.180000e+02
Name: emoji_count, dtype: float64

In [69]:
merged['emoji_count'].max()

118

In [70]:
merged.columns

Index(['external_author_id', 'author', 'content', 'region', 'language',
       'publish_date', 'following', 'followers', 'updates', 'is_retweet',
       'account_category', 'tweet_id', 'full_url', 'data_source', 'has_URL',
       'emoji_text', 'emoji_count'],
      dtype='object')

In [82]:
os.getcwd()

'/Users/katymatulay/Documents/Drexel - Grad School/07 Fall 2022/DSCI591/Capstone/data'

In [72]:
merged.head()

,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url,data_source,has_URL,emoji_text,emoji_count
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,1052,9636,253,0,RightTroll,914580356430536707,https://twitter.com/10_gop/status/914580356430...,Troll,1,[],0
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,1054,9637,254,0,RightTroll,914621840496189440,https://twitter.com/damienwoody/status/9145685...,Troll,1,[],0
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,1054,9637,255,1,RightTroll,914623490375979008,https://twitter.com/10_gop/status/913231923715...,Troll,1,[],0
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,1062,9642,256,0,RightTroll,914639143690555392,https://twitter.com/10_gop/status/914639143690...,Troll,1,[],0
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,1050,9645,246,1,RightTroll,914312219952861184,https://twitter.com/realDonaldTrump/status/914...,Troll,1,[flag: United States],1


In [75]:
merged['is_retweet'].value_counts()

0    1673960
1     936092
Name: is_retweet, dtype: int64

In [79]:
merged.dtypes

external_author_id    object
author                object
content               object
region                object
language              object
publish_date          object
following              int64
followers              int64
updates                int64
is_retweet             int64
account_category      object
tweet_id              object
full_url              object
data_source           object
has_URL                int64
emoji_text            object
emoji_count            int64
dtype: object

In [ ]:
###Convert publish_date to datetime object
merged['publish_date']= pd.to_datetime(merged['publish_date'], infer_datetime_format=True)

In [81]:
#writing to csv for ease of retrieving later, corrupt file is created, do not use
#merged.to_csv("merged_dataset2.csv",index=False, header= True,encoding='utf-8', float_format= '%.0f')

In [ ]:
#writing to pickle as CSV has issues with formatting and datatypes
merged.to_pickle("merged_datasetpickle.pkl")

In [85]:
#validate that it pickled correctly
dt = pd.read_pickle("../data/merged_datasetpickle.pkl")
dt.head()

,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url,data_source,has_URL,emoji_text,emoji_count
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,1052,9636,253,0,RightTroll,914580356430536707,https://twitter.com/10_gop/status/914580356430...,Troll,1,[],0
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,1054,9637,254,0,RightTroll,914621840496189440,https://twitter.com/damienwoody/status/9145685...,Troll,1,[],0
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,1054,9637,255,1,RightTroll,914623490375979008,https://twitter.com/10_gop/status/913231923715...,Troll,1,[],0
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,1062,9642,256,0,RightTroll,914639143690555392,https://twitter.com/10_gop/status/914639143690...,Troll,1,[],0
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,1050,9645,246,1,RightTroll,914312219952861184,https://twitter.com/realDonaldTrump/status/914...,Troll,1,[flag: United States],1
